This web crawler is to compare different prices of a book on two different website (books.com and kingstone.com) to help customer to decide where to buy it. The title, ISBN, different prices of the book will be saved to a csv file.

In [1]:
URL = 'https://search.books.com.tw/search/query/key/{0}/cat/all'

In [2]:
def generate_search_url(url, keyword):
    url = url.format(keyword)
    return url

In [3]:
import requests
import random
user_agents = [
 "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; AcooBrowser; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
 "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Acoo Browser; SLCC1; .NET CLR 2.0.50727; Media Center PC 5.0; .NET CLR 3.0.04506)",
 "Mozilla/4.0 (compatible; MSIE 7.0; AOL 9.5; AOLBuild 4337.35; Windows NT 5.1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
 "Mozilla/5.0 (Windows; U; MSIE 9.0; Windows NT 9.0; en-US)",
 "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 2.0.50727; Media Center PC 6.0)",
 "Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 1.0.3705; .NET CLR 1.1.4322)",
 "Mozilla/4.0 (compatible; MSIE 7.0b; Windows NT 5.2; .NET CLR 1.1.4322; .NET CLR 2.0.50727; InfoPath.2; .NET CLR 3.0.04506.30)",
 "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN) AppleWebKit/523.15 (KHTML, like Gecko, Safari/419.3) Arora/0.3 (Change: 287 c9dfb30)",
 "Mozilla/5.0 (X11; U; Linux; en-US) AppleWebKit/527+ (KHTML, like Gecko, Safari/419.3) Arora/0.6",
 "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.8.1.2pre) Gecko/20070215 K-Ninja/2.1.1",
 "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN; rv:1.9) Gecko/20080705 Firefox/3.0 Kapiko/3.0",
 "Mozilla/5.0 (X11; Linux i686; U;) Gecko/20070322 Kazehakase/0.4.5",
 "Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.8) Gecko Fedora/1.9.0.8-1.fc10 Kazehakase/0.5.6",
 "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11",
 "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/535.20 (KHTML, like Gecko) Chrome/19.0.1036.7 Safari/535.20",
 "Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; fr) Presto/2.9.168 Version/11.52",
    ]
def get_resource(url):
    headers = {"user-agent":random.choice(user_agents)}
    return requests.get(url, headers=headers)

In [36]:
from bs4 import BeautifulSoup

def parse_html(r):
    if r.status_code == requests.codes.ok:
        r.encoding = 'utf-8'
        soup = BeautifulSoup(r.text,'lxml')
    else:
        print('error')
        soup = None
    return soup

In [5]:
def get_ISBN(url):
    book_number = url.split('/')[10]
    new_url = 'https://www.books.com.tw/products/{0}?sloc=main'.format(book_number)
    soup = parse_html(get_resource(new_url))
    if soup != None:
        return soup.find(class_='mod_b type02_m058 clearfix').find('li').text[5:]
    else:
        return None
    

In [75]:
def get_another_price(isbn):
    url = 'https://www.kingstone.com.tw/search/key/'+isbn
    soup = parse_html(get_resource(url))
    # On kingstone website, if the isbn code cannot be found, it would give u several irrelavent books.
    # Only when the buymixbox is unique, the kingstone website gives the correct book.
    if len(soup.find_all(class_ = 'buymixbox')) == 1:
        tag_div = soup.find(class_ = 'buymixbox')
        price = tag_div.find_all('span')[1].b.text
    else:
        price=None
    return price

In [80]:
import time

def web_scraping_bot(url):
    booklist = []
    soup = parse_html(get_resource(url))
    if soup != None:
        tag_table = soup.find_all(class_='table-td')
        for item in tag_table:
            book = []
            book.append(item.find('img')['alt'])
            isbn = get_ISBN(item.find('a')['href'])
            book.append(isbn)
            if len(item.find(class_='price clearfix').find_all('b')) == 1:
                book.append(item.find(class_='price clearfix').find_all('b')[0].text)
            else:
                book.append(item.find(class_='price clearfix').find_all('b')[1].text)
            price= get_another_price(isbn)
            book.append(price)
            booklist.append(book)
            print('wait 10 seconds for next book...')
            time.sleep(10) #the website block too intensive scrapping
#             print(booklist)
    return booklist

In [62]:
import csv

def save_to_csv(booklist, file):
    with open(file, 'w+', newline='') as fp:
        writer = csv.writer(fp)
        for book in booklist:
            writer.writerow(book)

In [81]:
if __name__ == '__main__':
    url = generate_search_url(URL, 'machine learning')
    booklist = web_scraping_bot(url)
    save_to_csv(booklist, 'booklist.csv')

wait 10 seconds for next book...
wait 10 seconds for next book...
wait 10 seconds for next book...
wait 10 seconds for next book...
wait 10 seconds for next book...
wait 10 seconds for next book...
wait 10 seconds for next book...
wait 10 seconds for next book...
wait 10 seconds for next book...
wait 10 seconds for next book...
wait 10 seconds for next book...
wait 10 seconds for next book...
wait 10 seconds for next book...
wait 10 seconds for next book...
wait 10 seconds for next book...
wait 10 seconds for next book...
wait 10 seconds for next book...
wait 10 seconds for next book...
wait 10 seconds for next book...
wait 10 seconds for next book...
wait 10 seconds for next book...
wait 10 seconds for next book...
wait 10 seconds for next book...
wait 10 seconds for next book...


In [82]:
booklist

[['Machine Learning', '9789811519697', '3750', '3577'],
 ['Machine Learning', '9789811519666', '3750', None],
 ['Machine Learning', '9786203846447', '2155', '2419'],
 ['Machine Learning', '9781652126508', '315', None],
 ['Machine Learning', '9781107096394', '6300', None],
 ['Machine Learning', '9783902613561', '8055', '6927'],
 ['Machine Learning', '9780070428072', '20675', '19724'],
 ['Machine Learning', '9781558601482', '3648', None],
 ['Machine Learning', '9789533070339', '8055', '6927'],
 ['Python, Data Science and Machine Learning: From Scratch to Productivity',
  '9789811215728',
  '4900',
  None],
 ['Financial Data Analytics with Machine Learning, Optimization and Statistics',
  '9781119863373',
  '2625',
  None],
 ['Graph-Powered Analytics and Machine Learning with Tigergraph: Driving Business Outcomes with Connected Data',
  '9781098106652',
  '2100',
  None],
 ['Advanced Analytics, Machine Learning, and AI in Biopharmaceutical Manufacturing Companies',
  '9781119567509',
  '1